In [ ]:
# TODO: NER
# TODO: Word Sense Disambiguation
# TODO: NEL
# TODO: Cannonicalisation
# TODO: Open IE
# TODO: Opinion Triples
# TODO: Opinion Analysis, Visualisation, Maps: Entities, Similarity Clustering
# TODO: Knowledge Graph
# TODO: Neo4j
# TODO: Operationalise persist Cloud DB + App interface

In [3]:
import stanza
import pandas as pd
import json

In [4]:
# Stanza Pipeline
#stanza.download("en")
#stanza.install_corenlp()
nlp = stanza.Pipeline(lang="en", processors="tokenize, mwt, pos, ner")

2023-01-08 14:19:08 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-01-08 14:19:09 WARNING: Can not find mwt: default from official model list. Ignoring it.
2023-01-08 14:19:09 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| ner       | ontonotes |

2023-01-08 14:19:09 INFO: Use device: cpu
2023-01-08 14:19:09 INFO: Loading: tokenize
2023-01-08 14:19:09 INFO: Loading: pos
2023-01-08 14:19:10 INFO: Loading: ner
2023-01-08 14:19:10 INFO: Done loading processors!


In [5]:
all_news = [json.loads(_) for _ in open("data/RSS_objects.jsonl")]

In [ ]:
entities = []

for news in all_news:
    doc = nlp(news["summary"])

    for sent in doc.sentences:
        entity_id = len(entities)

        for ent in sent.ents:
            for _ in entities:
                found = False

                if _['text'] == ent.text and _['type'] == ent.type:
                    found = True
                    _["news_ids"].append(news["id"])
                    _["count"] += 1

                if 'entities' in news:
                    news['entities'].append(_['id'])

                else:
                    news['entities'] = [_['id']]

            if not found:
                entity = {
                    "id": entity_id,
                    "text": ent.text,
                    "type": ent.type,
                    "count": 1,
                    "news_ids": [news["id"]]

                }

                entities.append(entity)

                if "entities" in news:
                    news["entities"].append(entity_id)

                else:
                     news["entities"] = [entity_id]

                entity_id = entity_id + 1

In [ ]:
entities

In [ ]:
df = pd.DataFrame(data=entities)
df

In [ ]:
import sparql

entity_types = ['PERSON','NORP', 'FAC', 'ORG', 'GPE', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART', 'LAW']

s = sparql.Service("http://dbpedia.org/sparql", "GET")

In [ ]:
for entity in entities:

    if entity["type"] in entity_types:
        searchText = entity['text'].replace('"', '')
        curQuery = 'SELECT DISTINCT * WHERE '
        curQuery = curQuery + '{ ?url rdfs:label "' + searchText + '"@en . '
        curQuery = curQuery + 'FILTER(STRSTARTS(str(?url), "http://dbpedia.org/resource/"))}'

        try:
            results = s.query(curQuery).fetchall()

        except:
            continue

        if len(results) > 0:
            for result in results:
                resourceName = sparql.unpack_row(result)[0]
                if 'Category:' not in resourceName and 'property' not in resourceName:
                    print(resourceName)
                    entity['dbpedia_Resource'] = resourceName

In [ ]:
# Spotlight
import requests
from IPython.core.display import display, HTML

spotlight_url = "http://api.dbpedia-spotlight.org/en/annotate"

headers = {
    "accept": "application/json"
}

for _ in all_news:
    params = {
        "text": _["title"],
        "confidence": 0.35
    }
    try:
        res = requests.get(spotlight_url, params=params, headers=headers)
        _["annotation"] = res.json()

    except:
        _["annotation"] = "NIL"
        continue

In [ ]:
all_news[1]

In [318]:
# TODO: Mask
# TODO: Advanced Meta Data

item = all_news[5]

def locate(text, surface_form):
    items = text.split()
    len_ = len(surface_form.split())

    for idx, _ in enumerate(items):
        if len_ > 1:
            surface_form = surface_form.split()[0]

        if _.startswith(surface_form):
            return (idx, idx + len_)

        else: None

count = 0
for item in all_news:
    count += 1

    if item["annotation"] == "NIL":
        continue

    else:

        link_text = item["title"].split()

        if "Resources" in item["annotation"]:
            for ent in item["annotation"]["Resources"]:
                surface_form = ent["@surfaceForm"]
                uri = ent["@URI"]

                try:
                    start, finish = locate(item["title"], surface_form)

                except:
                    continue

                if start != finish:
                    # Slice Assignment:
                    link_text[start: finish] = [f"[{surface_form}, {uri}]"]

                elif start == finish:
                    link_text[start] = f"[{surface_form}, {uri}]"

            disambiguated = " ".join(_ for _ in link_text)
            item["NEL_text"] = disambiguated
            print(item["title"])
            print(disambiguated, "\n")

        else: continue

Twenty-Two Highlights from TRON’s Historic 2022
Twenty-Two Highlights from [TRON, http://dbpedia.org/resource/Tron] Historic 2022 

Twenty-Two Highlights from TRON’s Historic 2022
Twenty-Two Highlights from [TRON, http://dbpedia.org/resource/Tron] Historic 2022 

Australian Bitcoin ATMs Demonstrate Lightning Network
[Australian, http://dbpedia.org/resource/Australians] [Bitcoin, http://dbpedia.org/resource/Bitcoin] [ATMs, http://dbpedia.org/resource/Automated_teller_machine] [Demonstrate, http://dbpedia.org/resource/Demonstrate_(song)] [Lightning Network, http://dbpedia.org/resource/Lightning_Network] 

The Bahamas Regulator Denies Requesting FTX To Manufacture New Tokens
[The Bahamas, http://dbpedia.org/resource/The_Bahamas] Regulator Denies Requesting FTX To Manufacture New Tokens 

Morocco Completed Crypto Regulation
[Morocco, http://dbpedia.org/resource/Morocco] Completed Crypto Regulation 

Lee Jung-Hoon, Former Bithumb Chair Acquitted In First Instance
[Lee, http://dbpedia.org/re

In [321]:
NEL = [_ for _ in all_news if "NEL_text" in _]

"linked: ", len(NEL), "all: ", len(all_news)

('linked: ', 184, 'all: ', 188)

In [322]:
NEL

[{'title': 'Twenty-Two Highlights from TRON’s Historic 2022',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': 'https://Blockchain.News/RSS/',
   'value': 'Twenty-Two Highlights from TRON’s Historic 2022'},
  'content': [{'type': 'text/html',
    'language': None,
    'base': 'https://Blockchain.News/RSS/',
    'value': ''}],
  'summary': '<img src="https://Blockchain.News/RSS/" /><br />For TRON DAO, 2022 was a year of historic growth. Here are 22 highlights from the fastest growing public blockchain’s record year<a href="https://Blockchain.News/news/twenty-two-highlights-from-trons-historic-2022"> (Read More)</a>',
  'summary_detail': {'type': 'text/html',
   'language': None,
   'base': 'https://Blockchain.News/RSS/',
   'value': '<img src="https://Blockchain.News/RSS/" /><br />For TRON DAO, 2022 was a year of historic growth. Here are 22 highlights from the fastest growing public blockchain’s record year<a href="https://Blockchain.News/news/twenty-two-highlig

In [323]:
# STORE RSS-NEL Processes Objects
import os
data_dir = "./data/processed/"

if not os.path.isdir(data_dir):
    os.makedirs("./data/processed/")

file = data_dir + "/" + "NEL_objects.jsonl"
count = 0

with open(file, "w") as f:
    for _ in NEL:
        f.write(json.dumps(_))
        f.write("\n")

In [93]:
from bs4 import BeautifulSoup
from nltk import sent_tokenize

for item in all_news:
    summary = item["summary"]
    soup = BeautifulSoup(summary)
    summary = soup.get_text()

    parsed_item = item["title"] + "." + " " + summary

    sentences = sent_tokenize(parsed_item)
    item["sentences"] = sentences

/Users/joshua_sheppard/PycharmProjects/news_graph.py/venv/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [101]:
sents = [_["sentences"] for _ in all_news]
assert len(sents) == len(all_news)

In [ ]:
# OPEN IE
from stanza.server import CoreNLPClient

annotated = []
with CoreNLPClient(annotators=["openie"], be_quiet=False) as client:
    for _ in all_news:
        _["openie"] = []

        for sent in _["sentences"]:
            ann = client.annotate(sent)
            for sentence in ann.sentence:
                for triple in sentence.openieTriple:
                    triple_obj = {
                        "subject": triple.subject,
                        "relation": triple.relation,
                        "object": triple.object,
                    }

                    _["openie"].append(triple_obj)

In [109]:
open_ie = [_["openie"] for _ in all_news]
assert len(open_ie) == len(all_news)

In [110]:
# STORE RSS-NEL Processes Objects
import os

data_dir = "./data/processed/"

if not os.path.isdir(data_dir):
    os.makedirs("./data/processed/")

file = data_dir + "/" + "OIE_objects.jsonl"
count = 0

with open(file, "w") as f:
    for _ in all_news:
        f.write(json.dumps(_))
        f.write("\n")

In [90]:
# from pprint import pprint
# annotated = annotated
#
# news_triples = []
# for ann in annotated:
#     for sentence in ann.sentence:
#         triples = []
#         for triple in sentence.openieTriple:
#             triple_obj = {
#                 "subject": triple.subject,
#                 "relation": triple.relation,
#                 "object": triple.object,
#             }
#
#             triples.append(triple_obj)
#
#         news_triples.append(triples)